In [1]:
import requests
from bs4 import BeautifulSoup
from tools.json_tool import *
from tools.llm_api import *

def get_market_news():
    # 设置请求头，模仿浏览器
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # 获取网页内容
    url = "https://finance.eastmoney.com/a/ccjdd.html"
    response = requests.get(url, headers=headers)  

    # 确保请求成功
    if response.status_code == 200:
        # 解析网页内容
        soup = BeautifulSoup(response.content, 'html.parser')
        # 查找所有名称为title的class
        news_items = soup.find_all(class_='title')
        news_all = []
        for item in news_items:
            # 提取和打印新闻相关信息，例如标题
            title = item.get_text()  
            news_all.append(title)
    else:
        print(f"Failed to retrieve content, status code: {response.status_code}")

    return news_all


def get_prompt(news_all):
    prompt = f'''
# 你的角色
你是一个有十年经验的专业财经研究员

# 你的任务
对下面几条新闻进行分类，按照分类-新闻列举出来

# 新闻如下：
"""
{news_all}
"""

# 工作流
- 首先对新闻进行分类
- 然后给出市场整体属于积极、中性还是消极的判断, 将判断结果以json格式输出
参考示例：
```json
{{
    "市场判断": "积极"
}}
```

现在，请你进行分类：
'''
    return prompt

ans = ""
for char in get_llm_answer(get_prompt(get_market_news()), model="glm-4"):
    ans += char
    print(char, end='', flush=True)
    
emo = get_json(ans)
print(f"\n\n当前AI市场情绪判断为：{emo['市场判断']}")


以下是对给定新闻的分类：

**证券分析类：**
- 中信证券：电子行业下半年重点关注端侧AI、出海增量、国产自立三大方向
- 光大证券：围绕电力体制改革 持续推荐电网板块
- 中国银河证券：在美联储观望中寻求资产配置机会
- 银河证券：苹果发布Apple Intelligence 端侧AI有望迎来爆发

**投资建议类：**
- 桂浩明：辨析上市公司四大风险 做理性投资人

**宏观经济政策类：**
- 《求是》杂志发表习近平总书记重要文章《开创我国高质量发展新局面》
- 央行发布重磅数据！M1和M2同比增速为何持续下行

**市场动态类：**
- 万亿茅台大跌 什么信号？科技VS白酒 谁是王者

**国际市场类：**
- 隔夜外盘：纳指连续五日创收盘新高 法国股市大跌
- “石油美元协议终结”刷屏朋友圈！真相究竟如何？

以下是对市场整体情绪的判断：

```json
{
    "市场判断": "中性"
}
```

这个判断是基于各类新闻的平衡性作出的。虽然有一些积极的证券分析和宏观经济政策的文章，但也有关注到市场风险和国内大型股票的价格波动。国际市场的动态也呈现了分歧，纳斯达克指数创新高但法国股市大跌。综合来看，市场情绪整体较为中性。

当前AI市场情绪判断为：中性
